In [1]:
import semanticdebugger
from semanticdebugger.debug_algs.cl_utils import _keep_first_answer
from semanticdebugger.debug_algs.distant_supervision.ds_utils import create_training_stream
from semanticdebugger.debug_algs.index_based.index_manager import RandomMemoryManger
from semanticdebugger.debug_algs.index_based.index_utils import get_bart_dual_representation
from semanticdebugger.models import run_bart
from semanticdebugger.models.utils import set_seeds, trim_batch
import torch

from scipy.stats.stats import describe
from semanticdebugger.debug_algs.cl_simple_alg import ContinualFinetuning
import random
from tqdm import tqdm
from semanticdebugger.debug_algs import run_lifelong_finetune
import os

# os.environ["CUDA_VISIBLE_DEVICES"]="1"


# torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

torch.cuda.set_device(1)


In [2]:
parser = run_lifelong_finetune.get_cli_parser()
args = parser.parse_args("") 

cl, data_args, base_model_args, debugger_args, logger = run_lifelong_finetune.setup_args(
        args)

10/13/2021 22:07:04 - INFO - semanticdebugger.debug_algs.run_lifelong_finetune - Namespace(adam_epsilon=1e-08, adapter_dim=32, append_another_bos=1, base_model_path='out/mrqa_naturalquestions_bart-base_0617v4/best-model.pt', base_model_type='facebook/bart-base', bug_stream_json_path='bug_data/mrqa_naturalquestions_dev.static_bug_stream.json', cl_method_name='simple_cl', current_thread_id=None, data_stream_json_path='bug_data/mrqa_naturalquestions_dev.data_stream.test.json', do_lowercase=False, ewc_gamma=1, ewc_lambda=0.5, example_encoder_name='roberta-base', freeze_embeds=False, gradient_accumulation_steps=1, index_rank_method='most_similar', indexing_args_path='exp_results/supervision_data/1012_dm_simple.train_args.json', indexing_method='bart_index', inference_query_size=1, init_memory_cache_path='bug_data/memory_key_cache.pkl', learning_rate=1e-05, local_adapt_lr=1e-05, max_grad_norm=0.1, max_input_length=888, max_output_length=50, max_timecode=-1, memory_key_encoder='facebook/bart-

10/13/2021 22:07:04 - INFO - semanticdebugger.debug_algs.run_lifelong_finetune - Namespace(adam_epsilon=1e-08, adapter_dim=32, append_another_bos=1, base_model_path='out/mrqa_naturalquestions_bart-base_0617v4/best-model.pt', base_model_type='facebook/bart-base', bug_stream_json_path='bug_data/mrqa_naturalquestions_dev.static_bug_stream.json', cl_method_name='simple_cl', current_thread_id=None, data_stream_json_path='bug_data/mrqa_naturalquestions_dev.data_stream.test.json', do_lowercase=False, ewc_gamma=1, ewc_lambda=0.5, example_encoder_name='roberta-base', freeze_embeds=False, gradient_accumulation_steps=1, index_rank_method='most_similar', indexing_args_path='exp_results/supervision_data/1012_dm_simple.train_args.json', indexing_method='bart_index', inference_query_size=1, init_memory_cache_path='bug_data/memory_key_cache.pkl', learning_rate=1e-05, local_adapt_lr=1e-05, max_grad_norm=0.1, max_input_length=888, max_output_length=50, max_timecode=-1, memory_key_encoder='facebook/bart-

10/13/2021 22:07:04 - INFO - transformers.tokenization_utils - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json from cache at /private/home/yuchenlin/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
10/13/2021 22:07:04 - INFO - transformers.tokenization_utils - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-merges.txt from cache at /private/home/yuchenlin/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


10/13/2021 22:07:04 - INFO - transformers.tokenization_utils - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json from cache at /private/home/yuchenlin/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
10/13/2021 22:07:04 - INFO - transformers.tokenization_utils - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-merges.txt from cache at /private/home/yuchenlin/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


In [3]:
!pwd

/private/home/yuchenlin/SemanticDebugger/semanticdebugger/notebooks


In [79]:
base_model_args.base_model_path = os.path.join("/private/home/yuchenlin/SemanticDebugger/", base_model_args.base_model_path)
print(base_model_args)
cl.load_base_model(base_model_args)

10/13/2021 23:12:19 - INFO - semanticdebugger.debug_algs.run_lifelong_finetune - Loading checkpoint from /private/home/yuchenlin/SemanticDebugger/out/mrqa_naturalquestions_bart-base_0617v4/best-model.pt for facebook/bart-base .....


Namespace(base_model_path='/private/home/yuchenlin/SemanticDebugger/out/mrqa_naturalquestions_bart-base_0617v4/best-model.pt', model_type='facebook/bart-base')
10/13/2021 23:12:19 - INFO - semanticdebugger.debug_algs.run_lifelong_finetune - Loading checkpoint from /private/home/yuchenlin/SemanticDebugger/out/mrqa_naturalquestions_bart-base_0617v4/best-model.pt for facebook/bart-base .....


10/13/2021 23:12:20 - INFO - transformers.configuration_utils - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-base/config.json from cache at /private/home/yuchenlin/.cache/torch/transformers/09f4fcaeaf785dd3b97b085d6e3510c7081f586ec8e75981683c6299c0f81d9d.e8d516ad807436d395effad8c2326786872659b7dd1210827ac67c761198a0eb
10/13/2021 23:12:20 - INFO - transformers.configuration_utils - Model config BartConfig {
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel",
    "BartForConditionalGeneration",
    "BartForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_head

10/13/2021 23:12:20 - INFO - transformers.configuration_utils - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-base/config.json from cache at /private/home/yuchenlin/.cache/torch/transformers/09f4fcaeaf785dd3b97b085d6e3510c7081f586ec8e75981683c6299c0f81d9d.e8d516ad807436d395effad8c2326786872659b7dd1210827ac67c761198a0eb
10/13/2021 23:12:20 - INFO - transformers.configuration_utils - Model config BartConfig {
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel",
    "BartForConditionalGeneration",
    "BartForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_head

10/13/2021 23:12:24 - INFO - semanticdebugger.debug_algs.run_lifelong_finetune - Loading checkpoint from /private/home/yuchenlin/SemanticDebugger/out/mrqa_naturalquestions_bart-base_0617v4/best-model.pt for facebook/bart-base ..... Done!
10/13/2021 23:12:24 - INFO - semanticdebugger.debug_algs.run_lifelong_finetune - Moving to the GPUs.


10/13/2021 23:12:24 - INFO - semanticdebugger.debug_algs.run_lifelong_finetune - Loading checkpoint from /private/home/yuchenlin/SemanticDebugger/out/mrqa_naturalquestions_bart-base_0617v4/best-model.pt for facebook/bart-base ..... Done!
10/13/2021 23:12:24 - INFO - semanticdebugger.debug_algs.run_lifelong_finetune - Moving to the GPUs.


In [110]:

import math
from scipy import spatial
from transformers.modeling_bart import _prepare_bart_decoder_inputs

def masked_mean(reps, masks):
    masks = masks.view(reps.size()[0], reps.size()[1], 1)
    # print(f"masks.size()={masks.size()}")
    masked_reps = reps * masks
    # print(f"masked_reps.size()={masked_reps.size()}")
    # print(masked_reps[:, -3, :])
    # print(masked_reps[:, -2, :])
    # print(masked_reps[:, -1, :])

    # print(f"masks.sum(dim=1)={masks.sum(dim=1)}")
    masked_reps_sum = masked_reps.sum(dim=1)
    # print(f"masked_reps_sum.size()={masked_reps_sum.size()}")
    length_reps = masks.sum(dim=1).view(masked_reps_sum.size()[0], 1)
    # print(f"length_reps.size()={length_reps.size()}")
    
    mean_reps = masked_reps_sum / length_reps
    # print(mean_reps.size())
    return mean_reps

def get_bart_dual_representation_v1(cl_trainer, bart_model, tokenizer, data_args, examples, return_all_hidden=False):
    examples_with_single_ans = _keep_first_answer(examples)
    data_manager, _ = cl_trainer.get_dataloader(data_args,
                                                    examples_with_single_ans,
                                                    mode="train",
                                                    is_training=False)
    all_vectors = []
    bart_model = bart_model if cl_trainer.n_gpu == 1 else bart_model.module
    bart_model.eval()
    all_hiddens = {"input_reps":[], "input_masks": [], "output_reps": [] , "output_masks": []}
    for batch in tqdm(data_manager.dataloader):
        # self.logger.info(f"len(batch)={len(batch)}")
        if cl_trainer.use_cuda:
            # print(type(batch[0]), batch[0])
            batch = [b.to(torch.device("cuda")) for b in batch]
        pad_token_id = tokenizer.pad_token_id
        batch[0], batch[1] = trim_batch(
            batch[0], pad_token_id, batch[1])
        batch[2], batch[3] = trim_batch(
            batch[2], pad_token_id, batch[3])

        # Encode the input text with BART-encoder
        input_ids = batch[0]
        input_attention_mask = batch[1]
        encoder_outputs = bart_model.model.encoder(
            input_ids, input_attention_mask)
        x = encoder_outputs[0] 
        # x = x[:, 0, :]
        x = masked_mean(x, input_attention_mask)
        input_vectors = x.detach().cpu().numpy()

        # self.logger.info(f"input_vectors.shape = {input_vectors.shape}")

        # Encode the output text with BART-decoder

        output_ids = batch[2]
        output_attention_mask = batch[3]

        decoder_input_ids, decoder_padding_mask, causal_mask = _prepare_bart_decoder_inputs(
            bart_model.model.config,
            input_ids,
            decoder_input_ids=output_ids,
            decoder_padding_mask=output_attention_mask,
            causal_mask_dtype=bart_model.model.shared.weight.dtype,
        )
        decoder_outputs = bart_model.model.decoder(
            decoder_input_ids,
            encoder_outputs[0],
            input_attention_mask,
            decoder_padding_mask,
            decoder_causal_mask=causal_mask,
            decoder_cached_states=None,
            use_cache=False
        )
        y = decoder_outputs[0]
        # y = y[:, 0, :]
        y = masked_mean(y, output_attention_mask)
        output_vectors = y.detach().cpu().numpy()

        # self.logger.info(f"output_vectors.shape = {output_vectors.shape}")

        # concatenate the vectors
        vectors = np.concatenate([input_vectors, output_vectors], axis=1)
        if return_all_hidden:
            all_hiddens["input_reps"] += list(encoder_outputs[0].detach().cpu().numpy())
            all_hiddens["output_reps"] += list(decoder_outputs[0].detach().cpu().numpy())

            all_hiddens["input_masks"] += list(input_attention_mask.detach().cpu().numpy())
            all_hiddens["output_masks"] += list(output_attention_mask.detach().cpu().numpy())
            
            
        # self.logger.info(f"vectors.shape = {vectors.shape}")
        all_vectors += list(vectors)
        
        del batch
        del encoder_outputs
        del decoder_outputs
    if return_all_hidden:    
        return all_hiddens
    else:
        return all_vectors

import numpy as np

examples = []
examples += [("I like my bird.", "asd", "test_0")]
examples += [("I love my bird.", "asd", "test_1")]
examples += [("I love my dog.", "asd", "test_2")]
examples += [("I have a dream!", "asd", "test_3")]
examples += [("asdf as sdafasd", "asd", "test_4")]
examples += [("asdf as sdafasd dddd aaaaaa zcxv asdf  asdfgasdfasd ", "asd", "test_5")]

vectors = get_bart_dual_representation_v1(cl, cl.base_model, cl.tokenizer, data_args, examples)

all_hiddens = get_bart_dual_representation_v1(cl, cl.base_model, cl.tokenizer, data_args, examples, return_all_hidden=True)
  
input_reps = all_hiddens["input_reps"]
output_reps = all_hiddens["output_reps"]
input_masks = all_hiddens["input_masks"]
output_masks = all_hiddens["output_masks"]

encoder_layer = torch.nn.TransformerEncoderLayer(d_model=768, nhead=8)
transformer_encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=1)
input_reps = torch.Tensor(input_reps)
input_masks = torch.BoolTensor(input_masks)
input_reps = input_reps.permute(1,0,2)
# input_masks = input_masks.permute(1,0)
# input_masks = input_masks.view(input_masks.size()[0], input_masks.size()[1], 1)
print(f"input_reps.size()={input_reps.size()}")
print(f"input_masks.size()={input_masks.size()}")

# src = torch.rand(10, 32, 512)
out = transformer_encoder(input_reps, src_key_padding_mask=input_masks)
final_out = out[:,0,:]
print(out.size())

# # using the vectors 

# vectors = np.array(vectors)

# def calculate_cosine_distance(a, b):
#     cosine_distance = float(spatial.distance.cosine(a, b))
#     return cosine_distance


# def calculate_cosine_similarity(a, b, span="full"):
#     length = len(a) 
#     if span == "left":
#         a = a[:length//2]
#         b = b[:length//2]
#     elif span == "right":
#         a = a[length//2:]
#         b = b[length//2:]
#     cosine_similarity = 1 - calculate_cosine_distance(a, b)
#     return cosine_similarity
# print(vectors)
# print(calculate_cosine_similarity(vectors[0], vectors[1]))
# print(calculate_cosine_similarity(vectors[0], vectors[2]))
# print(calculate_cosine_similarity(vectors[0], vectors[3]))
# print(calculate_cosine_similarity(vectors[0], vectors[4])) 

100%|██████████| 1/1 [00:00<00:00, 37.48it/s]


input_reps.size()=torch.Size([29, 6, 768])
input_masks.size()=torch.Size([6, 29])
torch.Size([29, 6, 768])
